[![Open In
Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shaivimalik/medicine_preprocessing-on-entire-dataset/blob/main/notebooks/04.ipynb)

# Introduction

In the preceding notebooks, we demonstrated the impact of data leakage
on a model’s performance on the test set and real-world data. In this
notebook, we will reproduce the results published in **Characterization
of Term and Preterm Deliveries using Electrohysterograms Signatures**[1]
without the data leakage error. Our goal here is to demonstrate the
correct way of preprocessing the dataset and discuss the changes in the
reported metrics upon rectification of the error.

### Objectives

-   Implement the described techniques and train SVM without data
    leakage errors.
-   Analyze and compare our results with those published in the paper
    and obtained in the previous notebook.

[1] M. U. Khan, S. Aziz, S. Ibraheem, A. Butt and H. Shahid,
“Characterization of Term and Preterm Deliveries using
Electrohysterograms Signatures,” 2019 IEEE 10th Annual Information
Technology, Electronics and Mobile Communication Conference (IEMCON),
Vancouver, BC, Canada, 2019, pp. 0899-0905, doi:
10.1109/IEMCON.2019.893629

# Retrieve the data & Generate Features

The **Term-Preterm EHG Database**[1] is a collection of EHG signals
obtained from 1997 to 2005 at the University Medical Centre Ljubljana,
Department of Obstetrics and Gynecology. Electrohysterograms signatures
are obtained by placing four electrodes on the abdomen of the mother.
The TPEGH DB consists of EHG records obtained from 262 women who had
full-term pregnancies and 38 whose pregnancies ended prematurely. Each
record is composed of three channels, recorded from 4 electrodes. The
differences in the electrical potentials of the electrodes were
recorded, producing 3 channels. Each record consists of two files, a
header file (.hea) containing information regarding the record and the
data file (.dat) containing signal data[2].

We’ll begin by acquiring the TPEGH DB (Term-Preterm ElectroHysteroGram
Database) and extracting relevant features for our model training. The
following cell will:

-   Clone the project repository
-   Download the TPEGH DB dataset
-   Install required dependencies

Note that the download may take some time depending on your internet
connection speed.

[1] Fele-Žorž, G., Kavšek, G., Novak-Antolič, Ž. et al. A comparison of
various linear and non-linear signal processing techniques to separate
uterine EMG records of term and pre-term delivery groups. Med Biol Eng
Comput 46, 911–922 (2008). https://doi.org/10.1007/s11517-008-0350-y

[2] Goldberger, A., Amaral, L., Glass, L., Hausdorff, J., Ivanov, P. C.,
Mark, R., … & Stanley, H. E. (2000). PhysioBank, PhysioToolkit, and
PhysioNet: Components of a new research resource for complex physiologic
signals. Circulation \[Online\]. 101 (23), pp. e215–e220.

In [ ]:
!git clone https://github.com/shaivimalik/medicine_preprocessing-on-entire-dataset.git
%cd medicine_preprocessing-on-entire-dataset
!pip install -r requirements.txt
!curl -O https://physionet.org/static/published-projects/tpehgdb/term-preterm-ehg-database-1.0.1.zip
!unzip term-preterm-ehg-database-1.0.1.zip

In [ ]:
!mkdir individual_features
!python3 EHG-Oversampling/experiments/all_features.py term-preterm-ehg-database-1.0.1/tpehgdb individual_features --study FeaturesKhan
!python3 EHG-Oversampling/experiments/process_feature_files.py individual_features ./
%cd notebooks

## Loading the features

In this section, we will load the dataset from the CSV files created in
the previous step.

We start by importing the required modules.

In [ ]:
import os
import imblearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, make_scorer

We load feature matrix (features) and labels (y) from CSV files. The
`head()` function displays the first few rows of each dataframe for a
quick overview.

In [ ]:
#Loading feature vectors
features=pd.read_csv(os.path.join('..','raw_features.csv'))
features.head()

In [ ]:
#Loading labels
y=pd.read_csv(os.path.join('..','target.csv'))
y.head()

In [ ]:
# Extracting features required for our study
khan_features = [
    'FeaturesJager_fmed_ch1', 'FeaturesJager_max_lyap_ch1',
    'FeaturesJager_sampen_ch1', 'FeaturesJager_fmed_ch2',
    'FeaturesJager_max_lyap_ch2', 'FeaturesJager_sampen_ch2',
    'FeaturesJager_fmed_ch3', 'FeaturesJager_max_lyap_ch3',
    'FeaturesJager_sampen_ch3',
 ]
generic_features=[ c for c in features.columns if 'FeaturesAcharya' in c and 'SampleEntropy' in c ]

# Extract the relevant features for the study
X = features[khan_features + generic_features]

# Display summary information about the selected features
X.info()

# SVM Classifier Training and Evaluation without Data Leakage

In this section, we will train and evaluate the SVM-FG model using
10-fold cross validation. The paper doesn’t report the hyperparameters
used to train the model. Therefore, we will use `GridSearchCV` to find
optimal hyperparameter values for our classifier. We will then report
the accuracy, error, sensitivity and specificity, along with the
corresponding hyperparameter values.

We begin by creating custom scoring functions for specificity and
sensitivity. After that, we define the range of values for
hyperparameters C and gamma. `GridSearchCV` will search within these
ranges to find the optimal value of each hyperparameter.

In [ ]:
# Define specificity and sensitivity scoring functions
def specificity_score(y_true, y_pred):
    return recall_score(y_true, y_pred, pos_label=0)
def sensitivity_score(y_true, y_pred):
    return recall_score(y_true, y_pred)

# Create scorers using make_scorer
specificity = make_scorer(specificity_score)
sensitivity = make_scorer(sensitivity_score)

# Define parameters
gamma_range = np.logspace(start=-5, stop=5, num=11, base=2)
C_range = np.logspace(start=-5, stop=5, num=11, base=10)
parameters = {'C': C_range, 'gamma': gamma_range}

# Define scoring metrics for grid search including accuracy, sensitivity, and specificity
scoring = {'accuracy':'accuracy','sensitivity':sensitivity,'specificity':specificity}

Departing from the methodology followed in the paper, we will not
oversample the entire dataset. Instead, we will use ADASYN[1] to
oversample each training set separately during 10-fold cross-validation,
leaving the test set unaltered. The `pipeline` in `imbalanced-learn`
handles this for us and oversamples the training set correctly.

[1] Haibo He, Yang Bai, E. A. Garcia and Shutao Li, “ADASYN: Adaptive
synthetic sampling approach for imbalanced learning,” 2008 IEEE
International Joint Conference on Neural Networks (IEEE World Congress
on Computational Intelligence), Hong Kong, 2008, pp. 1322-1328, doi:
10.1109/IJCNN.2008.4633969. keywords: {Classification
algorithms;Decision trees;Algorithm design and analysis;Training
data;Machine learning;Accuracy;Machine learning algorithms}

In [ ]:
# Define the pipeline
model = imblearn.pipeline.Pipeline([
        ('ADASYN', imblearn.over_sampling.ADASYN(random_state=5)),
        ('SVM', SVC(kernel='rbf', random_state=5))
    ])

# Define the parameter grid for GridSearchCV
parameters={'SVM__C': C_range, 'SVM__gamma': gamma_range}

# Define GridSearchCV with custom scorers
clf = GridSearchCV(model, parameters, cv=10, scoring=scoring, refit='accuracy')

# Perform grid search
clf.fit(X.to_numpy(), y.to_numpy())

# Print results
print("Accuracy:", clf.best_score_)
print("Error:", (1-clf.best_score_))
print("Sensitivity:", clf.cv_results_['mean_test_sensitivity'][clf_cor.best_index_])
print("Specificity:", clf.cv_results_['mean_test_specificity'][clf_cor.best_index_])
print("Best hyperparameters:", clf.best_params_)

This code cell creates a heatmap to visualise the performance of the SVM
model. It plots the mean test accuracy obtained for different
combinations of hyperparamters C and gamma during `GridSearchCV`. The
x-axis represents different gamma values, and the y-axis represents
different C values. The colorbar on the side provides a scale for
interpreting the scores.

In [ ]:
# Set the figure size
plt.figure(figsize=(10, 8))
# Reshape the mean test accuracy scores into a 2D array
scores = clf.cv_results_["mean_test_accuracy"].reshape(len(C_range), len(gamma_range))
# Display the scores as a heatmap
plt.imshow(scores, interpolation="nearest", cmap=plt.cm.hot)
# Set the x-axis label
plt.xlabel("gamma")
# Set the y-axis label
plt.ylabel("C")
# Display the colorbar
plt.colorbar()
# Set the x-axis ticks and labels
plt.xticks(np.arange(gamma_range.shape[0]), labels=gamma_range, rotation=45)
# Set the y-axis ticks and labels
plt.yticks(np.arange(gamma_range.shape[0]), labels=C_range)
# Set the title of the plot
plt.title("Validation accuracy")
# Display the plot
plt.show()

# Discussion

It is evident from the results that the model trained with data leakage
has achieved higher accuracy than the model trained without data
leakage. This verifies that data leakage leads to overly optimistic
estimates of model performance. However, it’s important to note that in
the model without data leakage, the test set distribution may be more
imbalanced, with a higher proportion of majority class samples. This
imbalance can boost the model’s performance, as it allows the model to
perform well on the majority class.

However, despite this advantage, the model without data leakage still
underperforms compared to the model with data leakage. This
underperformance highlights the significant impact of data leakage in
inflating model performance. If we were to oversample the test set to
balance the classes for a fairer comparison, the accuracy of the model
without data leakage would likely decrease, as the increased
representation of the minority class would present a more challenging
scenario. Consequently, the gap between the accuracy reported in our
results and the published accuracy would widen even further. Thus, the
observed ~10% increase in accuracy can be considered as the lower limit
of the performance enhancement produced by data leakage in this case.

| Metric | Original | Reproduced With Data Leakage | Reproduced Without Data Leakage |
|:-----------:|:-------:|:-----------------------:|:------------------------:|
| Accuracy | 95.5 | 99.81 | 87.26 |
| Error | 4.48 | 0.19 | 12.74 |
| Specificity | 97.13 | 99.62 | 0.0 |
| Sensitivity | 93.51 | 100.0 | 100.0 |

In conclusion, we should always split our dataset prior to
preprocessing. This ensures integrity and reproducibility of our
results.